In [87]:
import re
import warnings
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import time
import fasttext
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

In [88]:

print("Loading data...")
path = "data/best_200songs_perartist_nw.csv"
data = pd.read_csv(path)
print("Data loaded. Shape:", data.shape)

# Filter lyrics for Taylor Swift
data = data[data['artist'] == 'Taylor Swift']
print("Data filtered for Taylor Swift. Shape:", data.shape)

# Further cleaning
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    return text

print("Preprocessing lyrics...")
data['clean_lyrics'] = data['clean_lyrics'].apply(preprocess_text)
print("Lyrics preprocessed. Sample data:\n", data['clean_lyrics'].head())

# Creating a corpus
print("Creating corpus...")
corpus = [lyrics.split() for lyrics in data['clean_lyrics']]
corpus_flat = [" ".join(sublist) for sublist in corpus]
with open("corpus.txt", "w") as file:
    file.write("\n".join(corpus_flat))
print("Corpus created. Total documents:", len(corpus))

# Train FastText model
print("Training FastText model...")
fasttext_model = fasttext.train_unsupervised('corpus.txt', model='skipgram', dim=100)
print("FastText model trained.")

# Creating vocabulary and mappings
print("Creating vocabulary and mappings...")
unique_words = fasttext_model.words
word_to_index = {word: idx for idx, word in enumerate(unique_words)}
index_to_word = {idx: word for word, idx in word_to_index.items()}
embedding_dim = fasttext_model.get_dimension()
print("Vocabulary size:", len(unique_words))

# Add a special token for unknown words
unk_token = '<UNK>'
word_to_index[unk_token] = len(word_to_index)
index_to_word[len(index_to_word)] = unk_token

Loading data...
Data loaded. Shape: (3800, 5)
Data filtered for Taylor Swift. Shape: (190, 5)
Preprocessing lyrics...
Lyrics preprocessed. Sample data:
 27    \n i walked through the door with you the air ...
31    \n vintage tee brand new phone \n high heels o...
48    \n we could leave the christmas lights up til ...
50    \n i m doing good i m on some new shit \n been...
65    \n i walked through the door with you the air ...
Name: clean_lyrics, dtype: object
Creating corpus...
Corpus created. Total documents: 190
Training FastText model...


Read 0M words
Number of words:  1169
Number of labels: 0
Progress:  63.7% words/sec/thread:  162579 lr:  0.018167 avg.loss:  2.593716 ETA:   0h 0m 0s

FastText model trained.
Creating vocabulary and mappings...
Vocabulary size: 1169


Progress: 100.0% words/sec/thread:  172235 lr:  0.000000 avg.loss:  2.608968 ETA:   0h 0m 0s


In [89]:
# Create an embedding matrix
print("Creating embedding matrix...")
embedding_matrix = np.zeros((len(unique_words) + 1, embedding_dim))
for word, idx in word_to_index.items():
    if word in fasttext_model:
        embedding_matrix[idx] = fasttext_model.get_word_vector(word)
    else:
        embedding_matrix[idx] = np.random.normal(size=(embedding_dim,))

# Prepare sequences for training
sequence_length = 40
features = []
targets = []
print("Preparing sequences for training...")
for lyrics in corpus:
    for i in range(len(lyrics) - sequence_length):
        input_sequence = [word_to_index.get(word, word_to_index[unk_token]) for word in lyrics[i:i+sequence_length]]
        target_word = word_to_index.get(lyrics[i+sequence_length], word_to_index[unk_token])
        features.append(input_sequence)
        targets.append(target_word)

print("Total sequences prepared:", len(features))

# Convert to numpy arrays
features = np.array(features)
targets = np.array(targets)
print("Converted sequences to numpy arrays. Features shape:", features.shape, "Targets shape:", targets.shape)

# Split the dataset into train and test sets
print("Splitting data into train and test sets...")
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)
print("Data split. Train shape:", X_train.shape, "Test shape:", X_test.shape)

# Convert to PyTorch tensors
print("Converting data to PyTorch tensors...")
X_train = torch.tensor(X_train, dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)
print("Data converted to tensors.")

Creating embedding matrix...
Preparing sequences for training...
Total sequences prepared: 66990
Converted sequences to numpy arrays. Features shape: (66990, 40) Targets shape: (66990,)
Splitting data into train and test sets...
Data split. Train shape: (53592, 40) Test shape: (13398, 40)
Converting data to PyTorch tensors...
Data converted to tensors.


In [91]:
class LyricsDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

print("Creating DataLoader...")
train_dataset = LyricsDataset(X_train, y_train)
test_dataset = LyricsDataset(X_test, y_test)

batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
print("DataLoader created.")

class LyricsLSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers, embedding_matrix):
        super(LyricsLSTMModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32), freeze=False)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)
        c0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# Hyperparameters
vocab_size = len(unique_words) + 1  # Add 1 for the UNK token
embed_size = embedding_dim
hidden_size = 256
num_layers = 2
num_epochs = 150
learning_rate = 0.001

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model = LyricsLSTMModel(vocab_size, embed_size, hidden_size, num_layers, embedding_matrix).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Creating DataLoader...
DataLoader created.


In [92]:
# Train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        start_time = time.time()
        total_loss = 0
        with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
                pbar.set_postfix({'loss': total_loss / (pbar.n + 1)})
                pbar.update(1)
        end_time = time.time()
        epoch_time = end_time - start_time
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Time: {epoch_time:.2f} sec')

print("Starting training...")
train_model(model, train_loader, criterion, optimizer, num_epochs)
print("Training completed.")

Starting training...


Epoch 1/150: 100%|██████████| 210/210 [00:16<00:00, 12.83batch/s, loss=5.56]


Epoch [1/150], Loss: 5.5561, Time: 16.37 sec


Epoch 2/150: 100%|██████████| 210/210 [00:13<00:00, 15.78batch/s, loss=5.26]


Epoch [2/150], Loss: 5.2635, Time: 13.31 sec


...

Epoch [148/150], Loss: 0.0270, Time: 21.23 sec


Epoch 149/150: 100%|██████████| 210/210 [00:21<00:00,  9.95batch/s, loss=0.192] 


Epoch [149/150], Loss: 0.1915, Time: 21.11 sec


Epoch 150/150: 100%|██████████| 210/210 [00:20<00:00, 10.00batch/s, loss=0.14] 

Epoch [150/150], Loss: 0.1399, Time: 20.99 sec
Training completed.


In [93]:
# Function to generate lyrics with temperature sampling and formatting
def generate_lyrics(model, start_text, length, temperature=1.0):
    model.eval()
    generated = start_text
    input_sequence = [word_to_index.get(word, word_to_index[unk_token]) for word in start_text.split()]
    generated_words = start_text.split()
    for _ in range(length):
        input_tensor = torch.tensor(input_sequence).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(input_tensor)
        output = output.squeeze().div(temperature).exp()
        word_weights = output.cpu().numpy()
        next_word_idx = np.random.choice(len(word_weights), p=word_weights/word_weights.sum())
        next_word = index_to_word[next_word_idx]
        generated_words.append(next_word)
        input_sequence.append(next_word_idx)
        input_sequence = input_sequence[1:]

    return generated_words

In [94]:
# Function to format lyrics into verses and choruses
def format_lyrics(generated_words):
    formatted_lyrics = "\nVerse 1:\n"
    line_length = 0
    verse_count = 1
    chorus_count = 1
    is_chorus = False

    for word in generated_words:
        formatted_lyrics += word + " "
        line_length += 1
        if line_length >= 7:
            formatted_lyrics = formatted_lyrics.strip() + "\n"
            line_length = 0
            if is_chorus:
                chorus_count += 1
                if chorus_count > 4:
                    is_chorus = False
                    chorus_count = 1
                    verse_count += 1
                    formatted_lyrics += f"\nVerse {verse_count}:\n"
            else:
                verse_count += 1
                if verse_count > 4:
                    is_chorus = True
                    verse_count = 1
                    formatted_lyrics += f"\nChorus:\n"

    return formatted_lyrics.strip()

# Example usage for lyrics generation
start_text = "Once upon a time"
generated_words = generate_lyrics(model, start_text, 72, temperature=1.2)
formatted_lyrics = format_lyrics(generated_words)
print("Generated lyrics:\n", formatted_lyrics)

Generated lyrics:
 Verse 1:
Once upon a time break anyone away
oh you had a sound like me
when let what that i i sweet
ta been to again had but away

Chorus:
i never did <UNK> to just you
and me little running soul shine thinking
knowing ll say is she we talk
of a dance need first face but

Verse 2:
we saw everyone is we ll gon
gon together you see the feeling every
time his way on a <UNK>
